Intro) Le but de ce projet est est de réaliser une analyse du marché des restaurants en France via Tripadvisor et de construire des dashboards synthetisant les informations obtenues.

Dans ce notebook, on met en forme les données et on crée une base de donnée MongoDB avec les données récupérées. Ces données seront ensuite utilisées par le notebook 4- Requetes Mongo pour produire des indicateurs, graphs sur le marché des restaurants en France

In [ ]:
import pandas as pd
import pymongo
import json
import ast
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [ ]:
#Connection à Mongo en Localhost
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["General"]
mycol = mydb["Restaurants_France"]

In [ ]:
#Fichier Excel contenant les données restaurants
df_restos = pd.read_excel(open('Restaurants_Tripadvisor.xlsx', 'rb'),sheet_name='Restos_France_2023')

In [ ]:
#Structuration des données et envoit sur la base
def mongo_pipe(row_number) :
    """
    Fonction prenant en paramètre un indice de ligne d'un dataframe
    et qui va mettre en forme et exporter sur une base de donnée MONGODB
    les données correspondant à la ligne spécifiée.
    
    Un traitement spécifique est réalisé sur les horaires d'ouvertures 
    pour adopter la structure suivante :
        - Le champ HORAIRES est un dictionnaire
        - Ce dictionnaire contient une liste pour chaque jour 
        durant lequel le restaurant est ouvert
        -Les listes de Jour contiennent chacune une liste par période
        consécutive d'ouverture. 
        - Les périodes d'ouvertures sont des listes, de 2 éléments
        indiquant l'heure à la quelle le restaurant ouvre et ferme
        
    Les périodes d'ouvertures tiennent également compte du passage 
    au jour suivant pour les établissements ouverts la nuit.
        
    Par exemple, un restaurant ouvert seulement le Lundi de 13h à 19h sera noté
    (schematiquement):
        - HORAIRES = ["Lun"]
        - ["Lun"] = ["1 ere periode d'ouverture du Lundi"]
        - ["1 ere periode d'ouverture du Lundi"] = [13h,19h ]
    
    Arguments:
        row_number : Integer : Indice de ligne d'un Dataframe
        
    Renvoit: Null
        Met en forme et exporte sur une base de donnée Mongo les 
        informations de la ligne en tant que nouveau document Mongo
    """
    url = df_restos["url"].iloc[row_number]
    nom = df_restos["nom"].iloc[row_number]
    adress = df_restos["adress"].iloc[row_number]
    ville = df_restos["Ville"].iloc[row_number]
    insee = df_restos["Insee"].iloc[row_number]
    code_departement = df_restos["Code dep"].iloc[row_number]
    epci = df_restos["EPCI"].iloc[row_number]
    departement = df_restos["Departement"].iloc[row_number]
    region = df_restos["Region"].iloc[row_number]
    #Sur Mongo, la longitude est indiquée en premier
    try :
        longitude = float(df_restos["gps"].iloc[row_number].split(",")[1])
        latitude = float(df_restos["gps"].iloc[row_number].split(",")[0])
        coordinates = [longitude,latitude]
    except:
        coordinates = [0,0]
    ranking = df_restos["Type"].iloc[row_number]
    note = df_restos["Note"].iloc[row_number]
    michelin = df_restos["Michelin"].iloc[row_number]
    if michelin == "MICHELIN" :
        michelin = True
    else:
        michelin = False
    note_michelin = df_restos["Note Michelin"].iloc[row_number]
    nb_avis = df_restos["Nb Avis"].iloc[row_number]
    categorie_prix = df_restos["Prix"].iloc[row_number]
    prix_fiable = df_restos["Garder_Prix"].iloc[row_number]
    if prix_fiable == "OUI" :
        prix_fiable = True
    else:
        prix_fiable = False
    methode_prix = df_restos["Methode"].iloc[row_number]
    prix_bas = df_restos["Prix Bas Euros Final"].iloc[row_number]
    prix_haut = df_restos["Prix Haut Euros Final"].iloc[row_number]
    try :
        cuisine = ast.literal_eval(df_restos["Cuisine"].iloc[row_number])
    except:
        cuisine = []
    titres_infos = ast.literal_eval(df_restos["Titres Details"].iloc[row_number])
    infos = ast.literal_eval(df_restos["Details"].iloc[row_number])
    for i,titre in enumerate(titres_infos) :
        if titre == "":
            del titres_infos[i]
            del infos[i]
    for i,bloc in enumerate(infos) :
        infos[i] = [info.replace("\u202f","") for info in bloc]

    titres_notes = ast.literal_eval(df_restos["Titres Details Notes"].iloc[row_number])
    notes = ast.literal_eval(df_restos["Details Notes"].iloc[row_number])
    #Mise en forme des horaires pour avoir une liste contenant un dictionnaire
    #par jour d'ouverture. Les dictionnaires de jour contiennent des listes,
    #1 liste par période d'ouverture continue. Ces Listes contiennent les heures
    #d'ouvertures et de fermetures en format timestamp
    try :
        format = '%H:%M'
        clean_horaires = []
        x = df_restos["horaires"].iloc[row_number].replace("Le matin","Matin").split(" ")
        x = x[1:]
        indexes = {}
        for day in ["dim","lun","mar","mer","jeu","ven","sam"] :
            try :
                indexes[day] = x.index(day)
            except Exception as e :
                pass
        values = list(indexes.values())
        horaires_resto = {}
        for i,key in enumerate(indexes.keys()) :
            horaires_resto[key.title()] = []
            try :
                subList = [x[n:n+5] for n in range(values[i]+1, values[i+1]-1, 5)]
            except IndexError:
                subList = [x[n:n+5] for n in range(values[i]+1, len(x), 5)]
            for plages_ouvertures in subList :
                horaire_ouverture = plages_ouvertures[0]
                periode_ouverture = plages_ouvertures[1]
                if (periode_ouverture == "L'après-midi" and horaire_ouverture[:2] !="12"):
                    horaire_ouverture = datetime.strptime(horaire_ouverture, format)+ timedelta(hours=12) 
                else:
                    horaire_ouverture = datetime.strptime(horaire_ouverture, format)

                horaire_fermeture = plages_ouvertures[3]
                periode_fermeture = plages_ouvertures[4]
                if periode_fermeture == "Matin"  and horaire_fermeture =="12:00" :
                    horaire_fermeture = datetime.strptime("00:00", format)+ timedelta(days=1) 
                elif periode_fermeture == "Matin"  and horaire_fermeture =="12:30" :
                    horaire_fermeture = datetime.strptime("00:30", format)+ timedelta(days=1)

                elif periode_fermeture == "L'après-midi" and horaire_fermeture !="12:00"and horaire_fermeture !="12:30":
                    horaire_fermeture = datetime.strptime(horaire_fermeture, format)+ timedelta(hours=12) 
                elif ((periode_fermeture == "Matin"  and periode_ouverture == "L'après-midi") 
                    or(horaire_ouverture >  datetime.strptime(horaire_fermeture, format) and periode_ouverture =="Matin")) :
                    horaire_fermeture = datetime.strptime(horaire_fermeture, format)+ timedelta(days=1) 
                else:
                    horaire_fermeture = datetime.strptime(horaire_fermeture, format)

                #Export données
                horaires_resto[key.title()].append([horaire_ouverture,horaire_fermeture])
    except :
        horaires_resto = []

    #Create dictionnary
    infos_clean = {titre:info for titre,info in zip(titres_infos,infos)}
    infos_clean.pop('', None)
    notes_clean = {titres_note:note for titres_note,note in zip(titres_notes,notes)}
    notes_clean.pop('', None)
    dic = {"Url":url,"Nom":nom,"Adresse":adress,"Ville":ville,"Code_Insee":insee,
            "Epci":epci,"Code_Departement":code_departement,"Departement":departement,
            "Region":region,"location": {"coordinates": coordinates,"type": "Point"},
            "Classement":ranking,"Note_Client":note,"Michelin":michelin,"Note_Michelin":note_michelin,
            "Nb_Avis":nb_avis,"Categorie_Prix":categorie_prix,"Prix_Fiable":prix_fiable,
            "Methode_Prix":methode_prix,"Prix_Bas":prix_bas,"Prix_Haut":prix_haut,
            "Cuisines":cuisine,"Details":infos_clean,"Details_Notes":notes_clean,"Horaires":horaires_resto}
    mycol.insert_one(dic)